In [1]:
!pip install -r requirements.txt

In [2]:
import json
import sys
import os
from openai import OpenAI
sys.path.append("src")

In [3]:
%load_ext autoreload
%autoreload 2

In [13]:
%reload_ext autoreload

In [5]:
from wiki_extract import Extracter
from wiki_gen import WikiGen

In [7]:
def get_article(name):
    print('Article name: ', name)
    page = Extracter(name)
    page.get_references()
    page.get_outline()
    page.get_reference_positions()
    page.newspaper_ref()
    page.get_filtered_outline()
    return page

In [8]:
p = get_article('Сколково (инновационный центр)')

Article name:  Сколково (инновационный центр)


Calculating reference positions: 100%|██████████████████████████████████████████████| 64/64 [00:00<00:00, 64605.40it/s]


In [14]:
p.get_text()
p.get_filtered_text()
print(len(p.text), len(p.filtered_text))

91494 53258


In [12]:
article_names = [
    'Python',
    'Летние Олимпийские игры 2024',
    'Квантовый компьютер',
    'Присоединение Крыма к Российской Федерации',
    'Сколково (инновационный центр)',
    'Tomb Raider (игра, 2013)',
    'Чёрная дыра',
    'Экономика США',
    'Искусственный интеллект',
    'COVID-19',
    'Броненосный крейсер'
]

In [6]:
def save_texts(article_name, texts, number):

    directory = os.path.join("Articles", "Sources", article_name)

    os.makedirs(directory, exist_ok=True)

    for i, text in enumerate(texts, start=number):
        file_path = os.path.join(directory, f"source_{i}.txt")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text)

In [13]:
for name in article_names:
    page = get_article(name)
    num = 1
    for _, texts in page.ref_texts.items():
        save_texts(page.name, texts, num)
        num += len(texts)

Article name:  Python


100%|██████████| 195/195 [21:19<00:00,  6.56s/it]


Article name:  Летние Олимпийские игры 2024


100%|██████████| 96/96 [01:59<00:00,  1.25s/it]


Article name:  Квантовый компьютер


...

100%|██████████| 89/89 [09:32<00:00,  6.43s/it]


Article name:  Искусственный интеллект


100%|██████████| 104/104 [08:59<00:00,  5.19s/it]


Article name:  COVID-19


Calculating reference positions: 100%|██████████| 57/57 [00:00<00:00, 56800.98it/s]
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
100%|██████████| 257/257 [26:02<00:00,  6.08s/it]


In [30]:
oclient = OpenAI(api_key=KEY, base_url=URL)

wiki_writer = WikiGen(oclient)

for name in article_names:

    json_str = wiki_writer.get_subqueries(name)

    if json_str:
        data = json.loads(json_str)
        directory = os.path.join("Generation", "Subqueries", name)
        os.makedirs(directory, exist_ok=True)
        filename = name.replace(" ", "_") + '.json'
        file_path = os.path.join(directory, filename)
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)